In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd
import numpy as np
import math
import optuna


import matplotlib.pyplot as plt

In [9]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.2 MB/s eta 0:00:00


In [11]:
stats = pd.read_csv('/content/drive/MyDrive/ELO/ufc_fight_stats.csv')
results = pd.read_csv('/content/drive/MyDrive/ELO/ufc_fight_results.csv')

In [12]:
# Extract fighter names from results BOUT column and assign to list
fighters = []
for i in results['BOUT']:
    fighters.append(i)
# Extract outcomes from results OUTCOME column and assign to list
outcome = []
for i in results["OUTCOME"]:
    outcome.append(i)
# Making panda daraframe with fighers and outcome and making it string values
df = pd.DataFrame(fighters, columns = ["Fighters"])
df['Fighters'] = df['Fighters'].astype('string')

df2 = pd.DataFrame(outcome, columns= ['Outcome'])
df2['Outcome'] = df2['Outcome'].astype('string')

# Spliting the first and second figher and making panda dataframe
first_name = []
second_name = []
for name in df["Fighters"]:
    first = name.split('  vs.')[0]
    second = name.strip()
    second = second.split('vs. ')[1]
    first_name.append(first)
    second_name.append(second)
df1 = pd.DataFrame(first_name, columns = ["Fighter_1"])
db = pd.DataFrame(second_name, columns = ["Fighter_2"])

# Splitting outcome for first and second fighter and maing panda dataframe
f1_out = []
f2_out = []
for i in df2['Outcome']:
    out = i.split('/')[0]
    out1 = i.split('/')[1]
    f1_out.append(out)
    f2_out.append(out1)
df3 = pd.DataFrame(f1_out, columns = ["F1_Out"])
db1 = pd.DataFrame(f2_out, columns = ["F2_Out"])

# Putting it all together
frames = [df1, db, df3, db1]
final = pd.concat(frames, axis = 1)
final['Fighter_1'] = final['Fighter_1'].astype('string')
final['Fighter_2'] = final['Fighter_2'].astype('string')
final['F1_Out'] = final['F1_Out'].astype('string')
final['Method'] = results['METHOD']

final.head(20)

,Fighter_1,Fighter_2,F1_Out,F2_Out,Method
0,Jared Cannonier,Nassourdine Imavov,L,W,KO/TKO
1,Dominick Reyes,Dustin Jacoby,W,L,KO/TKO
2,Raul Rosas Jr.,Ricky Turcios,W,L,Submission
3,Brunno Ferreira,Dustin Stoltzfus,W,L,KO/TKO
4,Julian Marquez,Zach Reese,L,W,KO/TKO
5,Miguel Baeza,Punahele Soriano,L,W,Decision - Unanimous
6,Thiago Moises,Ludovit Klein,L,W,Decision - Unanimous
7,Charles Radtke,Carlos Prates,L,W,KO/TKO
8,Brad Katona,Jesse Butler,W,L,Decision - Unanimous
9,Andrea Lee,Montana De La Rosa,L,W,Decision - Split


In [7]:
# Define ranges for k values
k_values_ranges = {
    'Decision - Unanimous': range(60, 121, 10),
    'KO/TKO': range(60, 101, 10),
    'Submission': range(60, 101, 10),
    'Decision - Split': range(10, 51, 10),
    'Decision - Majority': range(10, 51, 10),
    'TKO - Doctor\'s Stoppage': range(30, 71, 10),
    'Overturned': range(1, 11, 2),
    'Could Not Continue': range(10, 31, 5),
    'DQ': range(1, 11, 2),
    'Other': range(1, 11, 2)
}

# Initialize Elo ratings
initial_elo = 1500

def get_elo(fighter, elo_ratings):
    return elo_ratings.get(fighter, initial_elo)

def expected_score(rating1, rating2):
    return 1 / (1 + 10 ** ((rating2 - rating1) / 400))

def update_ratings(rating1, rating2, outcome, k):
    expected1 = expected_score(rating1, rating2)
    expected2 = expected_score(rating2, rating1)

    if outcome == 'W':
        score1, score2 = 1, 0
    else:
        score1, score2 = 0, 1

    new_rating1 = rating1 + k * (score1 - expected1)
    new_rating2 = rating2 + k * (score2 - expected2)

    return new_rating1, new_rating2

# Generate all possible combinations of k values
keys, values = zip(*k_values_ranges.items())
k_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

best_accuracy = 0
best_k_values = None
accuracy_results = []

# Iterate over each combination of k values
for k_values in k_combinations:
    elo_ratings = {}
    correct_predictions = 0
    total_predictions = 0

    # Process each fight in reverse order
    for index, row in final[::-1].iterrows():
        f1 = row['Fighter_1']
        f2 = row['Fighter_2']
        f1_out = row['F1_Out']
        method = str(row['Method']).strip()

        f1_rating = get_elo(f1, elo_ratings)
        f2_rating = get_elo(f2, elo_ratings)

        expected1 = expected_score(f1_rating, f2_rating)
        expected2 = expected_score(f2_rating, f1_rating)

        predicted_winner = f1 if expected1 > expected2 else f2
        actual_winner = f1 if f1_out == 'W' else f2

        if predicted_winner == actual_winner:
            correct_predictions += 1
        total_predictions += 1

        k = k_values.get(method, 200)

        new_f1_rating, new_f2_rating = update_ratings(f1_rating, f2_rating, f1_out, k)

        elo_ratings[f1] = new_f1_rating
        elo_ratings[f2] = new_f2_rating

    accuracy = correct_predictions / total_predictions
    accuracy_results.append((k_values, accuracy))

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k_values = k_values

print(f"Best k values: {best_k_values}")
print(f"Highest Accuracy: {best_accuracy * 100:.2f}%")

KeyboardInterrupt: 

In [15]:
# Initialize Elo ratings
initial_elo = 1500

def get_elo(fighter, elo_ratings):
    return elo_ratings.get(fighter, initial_elo)

def expected_score(rating1, rating2):
    return 1 / (1 + 10 ** ((rating2 - rating1) / 400))

def update_ratings(rating1, rating2, outcome, k):
    expected1 = expected_score(rating1, rating2)
    expected2 = expected_score(rating2, rating1)

    if outcome == 'W':
        score1, score2 = 1, 0
    else:
        score1, score2 = 0, 1

    new_rating1 = rating1 + k * (score1 - expected1)
    new_rating2 = rating2 + k * (score2 - expected2)

    return new_rating1, new_rating2

def objective(trial):
    k_values = {
        'Decision - Unanimous': trial.suggest_int('k_unanimous', 10, 200, step=5),
        'KO/TKO': trial.suggest_int('k_kotko', 10, 200, step=5),
        'Submission': trial.suggest_int('k_submission', 10, 200, step=5),
        'Decision - Split': trial.suggest_int('k_split', 10, 200, step=5),
        'Decision - Majority': trial.suggest_int('k_majority', 10, 200, step=5),
        'TKO - Doctor\'s Stoppage': trial.suggest_int('k_doctor', 10, 200, step=5),
        'Overturned': trial.suggest_int('k_overturned', 1, 31, step=2),
        'Could Not Continue': trial.suggest_int('k_continue', 10, 60, step=5),
        'DQ': trial.suggest_int('k_dq', 1, 11, step=2),
        'Other': trial.suggest_int('k_other', 1, 11, step=2)
    }

    elo_ratings = {}
    correct_predictions = 0
    total_predictions = 0

    # Process each fight in reverse order
    for index, row in final[::-1].iterrows():
        f1 = row['Fighter_1']
        f2 = row['Fighter_2']
        f1_out = row['F1_Out']
        method = str(row['Method']).strip()

        f1_rating = get_elo(f1, elo_ratings)
        f2_rating = get_elo(f2, elo_ratings)

        expected1 = expected_score(f1_rating, f2_rating)
        expected2 = expected_score(f2_rating, f1_rating)

        predicted_winner = f1 if expected1 > expected2 else f2
        actual_winner = f1 if f1_out == 'W' else f2

        if predicted_winner == actual_winner:
            correct_predictions += 1
        total_predictions += 1

        k = k_values.get(method, 200)

        new_f1_rating, new_f2_rating = update_ratings(f1_rating, f2_rating, f1_out, k)

        elo_ratings[f1] = new_f1_rating
        elo_ratings[f2] = new_f2_rating

    accuracy = correct_predictions / total_predictions
    return accuracy

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best set of hyperparameters
print(f"Best k values: {study.best_params}")
print(f"Highest Accuracy: {study.best_value * 100:.2f}%")

# Visualize the optimization process
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_param_importances(study).show()

[I 2024-07-09 13:51:04,209] A new study created in memory with name: no-name-6f72bcc6-126e-4716-b3a2-e42eec38d855
[I 2024-07-09 13:51:04,707] Trial 0 finished with value: 0.5378096479791395 and parameters: {'k_unanimous': 40, 'k_kotko': 95, 'k_submission': 60, 'k_split': 85, 'k_majority': 100, 'k_doctor': 75, 'k_overturned': 29, 'k_continue': 25, 'k_dq': 11, 'k_other': 3}. Best is trial 0 with value: 0.5378096479791395.
[I 2024-07-09 13:51:05,193] Trial 1 finished with value: 0.5488917861799217 and parameters: {'k_unanimous': 120, 'k_kotko': 190, 'k_submission': 160, 'k_split': 125, 'k_majority': 30, 'k_doctor': 175, 'k_overturned': 15, 'k_continue': 50, 'k_dq': 11, 'k_other': 9}. Best is trial 1 with value: 0.5488917861799217.
[I 2024-07-09 13:51:05,691] Trial 2 finished with value: 0.5372881355932203 and parameters: {'k_unanimous': 90, 'k_kotko': 45, 'k_submission': 110, 'k_split': 20, 'k_majority': 185, 'k_doctor': 30, 'k_overturned': 21, 'k_continue': 20, 'k_dq': 11, 'k_other': 9}.

Best k values: {'k_unanimous': 100, 'k_kotko': 180, 'k_submission': 120, 'k_split': 25, 'k_majority': 70, 'k_doctor': 20, 'k_overturned': 27, 'k_continue': 10, 'k_dq': 3, 'k_other': 7}
Highest Accuracy: 56.15%
